<br> Исходя из условий поставленной задачи выбор модели ограничивался только OpenSource проектами. \
<br> Выбор был между семейством Llama, Gemma, Mistral, phi, qwen. Учитывая ограниченность вычислительных мощностей и тем, что в будущем нам придется модель дообучать. Выбор сокращается до Gemma2-2b и моделей Llama2-7b и ниже. \
<br> Вполне логичным будет модель Gemma2-2b ввиду лучшего перфоманса по сравнению с Llama2, хорошей документации и быстрой производительности на слабых устройствах. Также ее будет проще дообучить ввиду меньшего количества параметров на последних слоях. \
<br> Модель отлично понимает английский + обучалась на веб документах, которые содержат разный контент, в том числе, контент по моей теме


Оценив результат синтеза датасета(которой вон там в csv файле) можно понять следующее: в датасете есть вопросы как по диете, по корректировке питания, так и вопросы по детоксу/гигиене/витаминам... Это произошло потому что правильное питание неразрывно связано с этими терминами и любой человек, когда начинает заниматься ЗОЖем/ПП встречается с ними

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


Касательно предобработки датасета - я его не делал 👍 по следующим причинам:
- вопросы генерируются агентом1, он уже убирает шум, плохие данные, ссылки и прочее..
- специфика тема не позволяет удалить, допустим, числа. так как они используются в рецетпах (сколько кг еды нужно) и в витаминах (люди могут спрашивать про содержание витаминов A, B в диетич. еде)
- после нормализации/лемматизации мы можем потерять важные связи в предложении и отвечать совсем на другой вопрос. (а механизм attention как раз позволяют все этим связи улавливать)

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

df = pd.read_csv('synthetic_dataset.csv')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=10)

train_dataset = Dataset.from_dict({
    'instruction': ['You are a great expert in the field of vegetarianism and recipes for vegans. You can see how recipes are built, which flavor combinations are the most popular.' for _ in range(len(train_df))],
    'input': train_df['question'].tolist(),
    'output': train_df['answer'].tolist()
})

test_dataset = Dataset.from_dict({
    'instruction': ['You are a great expert in the field of vegetarianism and recipes for vegans. You can see how recipes are built, which flavor combinations are the most popular.' for _ in range(len(test_df))],
    'input': test_df['question'].tolist(),
    'output': test_df['answer'].tolist()
})

In [ ]:
alpaca_prompt = """Below is an instruction that describe  s your role, paired with an input that provides question you need to answer. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN  # without this token generation goes on forever!
        texts.append(text)
    return {"text": texts}

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
test_dataset = test_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Так как в целом нейросеть уже имела представление о датасете и следует просто эти знания подкорректировать, то я выбрал следующие гиперпараметры (в TrainingArguments) \
<br> Там же я использую средства по предотвращению переобучения - weight_decay, так как без него мы начинаем переобучаться

<br> Большое влияние имеют количество эпох (что неудивительно) и learning_rate. После 6 запусков я оставил лучшие параметры. (num_epochs=2 не хватало и мы получали оценку 0.71.., а num_epochs=4 уже было много, получали 0.73). Как я понял, это стандартная практика ставит 1-3 эпохи для дообучения.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=512,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

Map (num_proc=2):   0%|          | 0/80 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 80 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,2.913000
2,3.119000
3,2.833500
4,2.550800
5,2.437100
6,2.223100
7,1.946600
8,1.832900
9,1.400100
10,1.332200


TrainOutput(global_step=30, training_loss=1.388383396466573, metrics={'train_runtime': 104.0159, 'train_samples_per_second': 2.307, 'train_steps_per_second': 0.288, 'total_flos': 763022048532480.0, 'train_loss': 1.388383396466573, 'epoch': 3.0})

<br> Благодаря величайшей библиотеке unsloth которая как-то волшебно играет с памятью, время дообучения - 1 минута 18 секунд (на T4, конечно). \
<br> Время инференса составило чуть больше 2х минут для 20 объектов. Это отличный результат. (различия с временем инференса обычной модели не наблюдаются)

In [ ]:
responses = []
FastLanguageModel.for_inference(model)
for i in tqdm(test_dataset):
    inputs = tokenizer([alpaca_prompt.format(
        i['instruction'], # instruction
        i['input'], # input
        "") ], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
    response = tokenizer.batch_decode(outputs)
    answ = response[0][response[0].index('Response:')+len('Response:')+1:]
    if '<eos>' in answ:
        answ = answ[:answ.index('<eos>')]
    responses.append(answ)

100%|██████████| 20/20 [02:03<00:00,  6.15s/it]


In [ ]:
actual_answers = []
for i in test_dataset:
    actual_answers.append(i['output'])

In [ ]:
%pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util
encoding = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
mean_score = []
for i in range(20):
    reference_embeddings = encoding.encode(actual_answers[i], convert_to_tensor=True)
    generated_embeddings = encoding.encode(responses[i], convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(generated_embeddings, reference_embeddings)
    mean_score.append(cosine_scores.diag().mean().item())

print('Mean score STS: ', np.array(mean_score).mean())

Mean score STS:  0.7510703101754188


Получаем усредненный результать Semantic Similarity 0.751. Это крутой результат, видно, что модель дообучилась. Обычная модель на тестовой выборке показала результат 0.70. Также важно обратить внимание на генерируемые ответы - они стали лаконичнее и точнее.

Так как процесс дообучения влияет на последние слои нн, то можно сделать очевидный вывод, что модель стала более применимой к нашим данным и к нашей задаче. Так как мы дообучаем модель на очень малом количестве данных, то ее предсказательные способности на другие темы не изменились

Далее будет простыня с сгенерированными ответами, просто решил ее оставить, после нее ничего не будет

In [ ]:
for i,v in enumerate(responses):
  print(i)
  print(v)
  print('---------------------------------------------------')

0
The recipe suggests cooking the stuffed roast in the pressure cooker for 45 minutes, but it's important to note that this is an estimate based on the size of the roast and the desired doneness.  Actual cooking times may vary depending on these factors.
---------------------------------------------------
1
There are various types of parasites, including bacteria, viruses, fungi, and protozoa. They can be transmitted through contaminated food, water, or direct contact with infected individuals.  These parasites can lead to various health issues, including digestive problems, weakened immune systems, and even more serious infections.
---------------------------------------------------
2
To ensure your family enjoys healthy meals while reducing calories, focus on flavor combinations that are naturally low in calories.  Use herbs, spices, and fresh vegetables to enhance the taste without adding extra fat or sugar.  Experiment with different flavor profiles to keep things interesting.
----

🥺